In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

2023-02-01 11:42:26.208615: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 11:42:26.698714: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-01 11:42:26.698733: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-01 11:42:28.412970: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [37]:
import random

# tensor = tf.cast(tf.one_hot([0,1,2,3,0,0,0,0,0,0,0,0,0,0,0,0], depth=4), tf.int32)
# tensor = tf.constant([[0,0,0,0]*20]*16)
tensor = tf.cast(tf.one_hot(random.choices([0,1,2,3], k=1024), depth=4), tf.int32)
print(tensor.shape)
print(tf.size(tensor))

(1024, 4)
tf.Tensor(4096, shape=(), dtype=int32)


In [38]:
sparse_tensor = tf.sparse.from_dense(tensor)
print(sparse_tensor.shape)
print(tf.size(sparse_tensor))
sparse_tensor

(1024, 4)
tf.Tensor(4096, shape=(), dtype=int32)


SparseTensor(indices=tf.Tensor(
[[   0    0]
 [   1    3]
 [   2    2]
 ...
 [1021    0]
 [1022    3]
 [1023    0]], shape=(1024, 2), dtype=int64), values=tf.Tensor([1 1 1 ... 1 1 1], shape=(1024,), dtype=int32), dense_shape=tf.Tensor([1024    4], shape=(2,), dtype=int64))

In [15]:
tf.io.serialize_sparse(sparse_tensor)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'\x08\t\x12\x06\x12\x00\x12\x02\x08\x02',
       b'\x08\x03\x12\x02\x12\x00',
       b'\x08\t\x12\x04\x12\x02\x08\x02"\x10\x10\x00\x00\x00\x00\x00\x00\x00P\x00\x00\x00\x00\x00\x00\x00'],
      dtype=object)>

In [8]:
isinstance(sparse_tensor, tf.SparseTensor)

True

In [18]:
d1 = {'a': 1, 'b': 2}
d2 = {'a': 3, 'c': 3}

d2 | d1

{'a': 1, 'c': 3, 'b': 2}

In [41]:
dataset = tf.data.Dataset.from_tensor_slices([tensor, tensor])
dataset = dataset.map(lambda x: {'abc': tf.sparse.from_dense(x)})
next(iter(dataset))

{'abc': SparseTensor(indices=tf.Tensor(
 [[ 0  0]
  [ 1  1]
  [ 2  2]
  [ 3  3]
  [ 4  0]
  [ 5  0]
  [ 6  0]
  [ 7  0]
  [ 8  0]
  [ 9  0]
  [10  0]
  [11  0]
  [12  0]
  [13  0]
  [14  0]
  [15  0]], shape=(16, 2), dtype=int64), values=tf.Tensor([1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1], shape=(16,), dtype=int32), dense_shape=tf.Tensor([16  4], shape=(2,), dtype=int64))}

In [42]:
dataset.element_spec

{'abc': SparseTensorSpec(TensorShape([16, 4]), tf.int32)}

In [5]:
def tensorspec_to_tensor(tensorspec):
    return tfds.features.Tensor(shape=tensorspec.shape, dtype=tensorspec.dtype)

def dataset_to_features(dataset):
    return tfds.features.FeaturesDict(tf.nest.map_structure(tensorspec_to_tensor, dataset.element_spec))

# dataset_to_features(dataset)

In [6]:
# class SparseTensor()

In [49]:
features_zlib = tfds.features.FeaturesDict({
    'a': tfds.features.Tensor(shape=(1024, 4), dtype=tf.int32, encoding='zlib')
})

features_bytes = tfds.features.FeaturesDict({
    'a': tfds.features.Tensor(shape=(1024, 4), dtype=tf.int32, encoding='bytes')
})

In [50]:
serialized_tensor = features_zlib.serialize_example({'a': tensor.numpy()})
print(len(serialized_tensor))
# print(serialized_tensor)

serialized_tensor = features_bytes.serialize_example({'a': tensor.numpy()})
print(len(serialized_tensor))
# print(serialized_tensor)

761
16407


In [51]:
serialized_sparse_tensor = tf.io.serialize_sparse(sparse_tensor).numpy()
print(len(serialized_sparse_tensor))
print(sum([len(s) for s in serialized_sparse_tensor]))
# print(serialized_sparse_tensor)

3
20535


In [33]:
len(serialized_sparse_tensor[0].numpy())

10

In [26]:
features.serialize_example({'a': sparse_tensor.numpy()})

AttributeError: 'SparseTensor' object has no attribute 'numpy'

In [16]:
tf.io.SparseFeature()

TypeError: SparseFeature.__new__() missing 4 required positional arguments: 'index_key', 'value_key', 'dtype', and 'size'

In [ ]:
tf.io.serialize_sparse()

In [21]:
import tensorflow as tf
import tensorflow_datasets as tfds


tesnor_feature = tfds.features.Tensor(shape=(None,), dtype=tf.int32)

In [22]:
tensor = tf.constant([1,2,3], dtype=tf.int32)

In [29]:
tesnor_feature.decode_example(tensor.numpy())

array([1, 2, 3], dtype=int32)

In [25]:
tensor.numpy()

array([1, 2, 3], dtype=int32)

In [48]:
def call_assert_output_type(type):
    def decorator(func):
        def wrapper(x):
            print(x)
            result = func(x)
            # assert isinstance(result, type)
            return result
        return wrapper
    return decorator

@call_assert_output_type(type=int)
def number(x):
    return x
print(number(7))

7
7


In [78]:
def times_factor(factor):
    def decorator(func):
        def wrapper(x):
            print(x)
            result = func(x)*factor
            return result
        return wrapper
    return decorator

class JustNumber():
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, number):
        return number

class JustNumberDeco():
    def __init__(self, factor):
        self.factor = factor
    
    @times_factor(factor=2)
    def __call__(self, number):
        return number

just_number_two = JustNumberDeco(2)
print(just_number_two(7))

TypeError: times_factor.<locals>.decorator.<locals>.wrapper() takes 1 positional argument but 2 were given

In [66]:
just_number = JustNumber(3)
f = just_number.__call__
f

<bound method JustNumber.__call__ of <__main__.JustNumber object at 0x7ff6744f4c70>>

In [68]:
JustNumber.__call__(just_number, 3)

3

In [77]:
type(JustNumber.__call__)

function

In [75]:
def print_hello_number(x):
    print('hello', x)

f = print_hello_number

In [76]:
type(f).__call__(f, 7)

hello 7


In [ ]:
def pyfunc_lambda_wrapper(func)

In [27]:
def _dtype_to_tensor_spec(v):
    return tf.TensorSpec(None, v) if isinstance(v, tf.dtypes.DType) else v

def _tensor_spec_to_dtype(v):
  return v.dtype if isinstance(v, tf.TensorSpec) else v

def py_function_nest(func, inp, Tout, name=None):
    def wrapped_func(*flat_inp):
        reconstructed_inp = tf.nest.pack_sequence_as(inp, flat_inp, expand_composites=True)
        out = func(*reconstructed_inp)
        return tf.nest.flatten(out, expand_composites=True)

    flat_Tout = tf.nest.flatten(Tout, expand_composites=True)
    flat_out = tf.py_function(
        func=wrapped_func, 
        inp=tf.nest.flatten(inp, expand_composites=True),
        Tout=[_tensor_spec_to_dtype(v) for v in flat_Tout],
        name=name)
    spec_out = tf.nest.map_structure(_dtype_to_tensor_spec, Tout, expand_composites=True)
    out = tf.nest.pack_sequence_as(spec_out, flat_out, expand_composites=True)
    return out

In [85]:
import tensorflow as tf

def times_factor(factor):
    def decorator(func):
        def wrapper(x):
            print(x)
            result = func(x)*factor
            # assert isinstance(result, type)
            return result
        return wrapper
    return decorator

# @times_factor(factor=2)
def number(x, **kwargs):
    return x

# print(number(7))

dataset = tf.data.Dataset.from_tensor_slices([1,2,3])
dataset = dataset.map(lambda *args, **kwargs: py_function_nest(number, inp=[args, kwargs], Tout=tf.int32)) # number_msg
print(next(iter(dataset)))

tf.Tensor(1, shape=(), dtype=int32)


In [90]:
def times_factor(factor):
    def decorator(func):
        def wrapper(*args, **kwargs):
            print(x)
            result = func(x)*factor
            # assert isinstance(result, type)
            return result
        return wrapper
    return decorator

class JustNumberDeco():
    def __init__(self, factor):
        self.factor = factor
    
    @times_factor(factor=2)
    def __call__(self, number, *args, **kwargs):
        return number

number = JustNumberDeco(2)

dataset = tf.data.Dataset.from_tensor_slices([1,2,3])
dataset = dataset.map(lambda *args, **kwargs: py_function_nest(number, inp=[args, kwargs], Tout=tf.int32)) # number_msg
print(next(iter(dataset)))

2023-02-01 11:30:11.066210: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: TypeError: times_factor.<locals>.decorator.<locals>.wrapper() takes 1 positional argument but 3 were given
Traceback (most recent call last):

  File "/home/marc/miniconda3/envs/bioflow/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    return func(device, token, args)

  File "/home/marc/miniconda3/envs/bioflow/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 147, in __call__
    outputs = self._call(device, args)

  File "/home/marc/miniconda3/envs/bioflow/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 154, in _call
    ret = self._func(*args)

  File "/home/marc/miniconda3/envs/bioflow/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/tmp/__autograph_generated_filet7lif9a3.py", line 17, in wrapped_func
    out = ag__.co

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: times_factor.<locals>.decorator.<locals>.wrapper() takes 1 positional argument but 3 were given
Traceback (most recent call last):

  File "/home/marc/miniconda3/envs/bioflow/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    return func(device, token, args)

  File "/home/marc/miniconda3/envs/bioflow/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 147, in __call__
    outputs = self._call(device, args)

  File "/home/marc/miniconda3/envs/bioflow/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 154, in _call
    ret = self._func(*args)

  File "/home/marc/miniconda3/envs/bioflow/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/tmp/__autograph_generated_filet7lif9a3.py", l

In [19]:
def _dtype_to_tensor_spec(v):
    return tf.TensorSpec(None, v) if isinstance(v, tf.dtypes.DType) else v

def _tensor_spec_to_dtype(v):
  return v.dtype if isinstance(v, tf.TensorSpec) else v

def py_function_nest(func, inp, Tout, name=None):
    def wrapped_func(*flat_inp):
        reconstructed_inp = tf.nest.pack_sequence_as(inp, flat_inp, expand_composites=True)
        out = func(*reconstructed_inp)
        return tf.nest.flatten(out, expand_composites=True)

    flat_Tout = tf.nest.flatten(Tout, expand_composites=True)
    flat_out = tf.py_function(
        func=wrapped_func, 
        inp=tf.nest.flatten(inp, expand_composites=True),
        Tout=[_tensor_spec_to_dtype(v) for v in flat_Tout],
        name=name)
    spec_out = tf.nest.map_structure(_dtype_to_tensor_spec, Tout, expand_composites=True)
    out = tf.nest.pack_sequence_as(spec_out, flat_out, expand_composites=True)
    return out

def tensor_to_numpy(tensor):
    return tensor.numpy()

def decode_if_bytes(x):
    if isinstance(x, bytes):
        x = x.decode('UTF-8')
    return x

def pyfunc(output_types, numpy=True, decode_bytes=True, expand_kwargs=True):
    def decorator(func):
        def func_wrapper(inp):
            if numpy:
                inp = tf.nest.map_structure(tensor_to_numpy, inp)
            if decode_bytes:
                inp = tf.nest.map_structure(decode_if_bytes, inp)
            
            if expand_kwargs:
                return func(**inp)
            else:
                return func(inp)
        return lambda x: py_function_nest(func_wrapper, inp=[x], Tout=output_types)
    return decorator

In [59]:
def pyfunc(Tout):
    def decorator(func):
        def wrapper(*args, **kwargs):
            result = py_function_nest(func, inp=[args, kwargs], Tout=Tout)
            return result
        return wrapper
    return decorator

class TimesNumber():
    def __init__(self, factor):
        self.factor = factor
    
    def multiply(self, a):
        return a * self.factor

    # def __call__(self, a, *args, **kwargs):
    #     return py_function_nest(lambda x: self.multiply(x), inp=[a], Tout=tf.int32)

    def __call__(self, kwargs):
        return py_function_nest(lambda kwargs: self.multiply(**kwargs), inp=[kwargs], Tout=tf.int32)

times_seven = TimesNumber(2)
# print(times_seven(a=2))

# @pyfunc(tf.int32)
# def wrapper(*args, **kwargs):
#     return number(*args, **kwargs)

# print(wrapper(7))

# def number(x, *args, **kwargs):
#     return x

# @pyfunc(Tout=tf.int32)
# def number_pyfunc(x, *args, **kwargs):
#     return x


# dataset = tf.data.Dataset.from_tensor_slices([2,3,4])
dataset = tf.data.Dataset.from_tensor_slices([2, 3, 4])
dataset = dataset.map(lambda x: {'a': x})
# # dataset = dataset.map(lambda *args, **kwargs: py_function_nest(number, inp=[args, kwargs], Tout=tf.int32)) # number_msg
dataset = dataset.map(times_seven)
for example in dataset:
    print(example)

tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)


In [24]:
import tensorflow as tf

d1 = tf.data.Dataset.from_tensor_slices([1, 2, 3])
d1 = d1.map(lambda x: {'a': {'n1': x}, 'i': 'd1'})
d1
x1 = next(iter(d1))

d2 = tf.data.Dataset.from_tensor_slices([7, 8, 9])
d2 = d2.map(lambda x: {'a': {'n2': x}, 'i': 'd2'})
d2
x2 = next(iter(d2))

In [25]:
dz = tf.data.Dataset.zip((d1, d2))
dz

<ZipDataset element_spec=({'a': {'n1': TensorSpec(shape=(), dtype=tf.int32, name=None)}, 'i': TensorSpec(shape=(), dtype=tf.string, name=None)}, {'a': {'n2': TensorSpec(shape=(), dtype=tf.int32, name=None)}, 'i': TensorSpec(shape=(), dtype=tf.string, name=None)})>

In [26]:
xl = (x1, x2)

In [29]:
def merge_dicts(dicts):
    xm = dicts[0]
    for x in dicts[1:]:
        xm = {**xm, **x}
    return xm

merge_dicts(xl)

{'a': {'n2': <tf.Tensor: shape=(), dtype=int32, numpy=7>},
 'i': <tf.Tensor: shape=(), dtype=string, numpy=b'd2'>}

In [30]:
dm = dz.map(lambda *dicts: merge_dicts(dicts))
dm

next(iter(dm))

{'a': {'n2': <tf.Tensor: shape=(), dtype=int32, numpy=7>},
 'i': <tf.Tensor: shape=(), dtype=string, numpy=b'd2'>}

In [31]:
def merge_nested_dicts(d1, d2, _nest_level=0):
    """Merges two dicts. 

    Args:
        d1 (dict): Dictionary 1. 
        d2 (dict): Dictionary 2. 
        _nest_level (int, optional): Private argument used for better error messages during recursion. Defaults to 0.

    Raises:
        ValueError: Raised if a duplicate atomic key is encountered at the same nesting level. 

    Returns:
        dict: Merged dictionary. 
    """

    for key in d2:
        if key not in d1:
            d1[key] = d2[key]
        elif isinstance(d1[key], dict) and isinstance(d2[key], dict):
            d1[key] = merge_nested_dicts(d1[key], d2[key], _nest_level=_nest_level+1)
    return d1

def merge_dicts_2(dicts):
    xm = dicts[0]
    for x in dicts[1:]:
        xm = merge_nested_dicts(xm, x)
    return xm

In [32]:
dm = dz.map(lambda *dicts: merge_dicts_2(dicts))
dm

next(iter(dm))

TypeError: in user code:

    File "/tmp/ipykernel_87154/1186788693.py", line 1, in None  *
        lambda *dicts: merge_nested_dicts(dicts)

    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__merge_nested_dicts() missing 1 required positional argument: 'd2'


In [65]:
dm = dz.map(lambda *args: {**x for x in args})
dm

next(iter(dm))

SyntaxError: dict unpacking cannot be used in dict comprehension (2050226427.py, line 1)

In [71]:
tf.nest.flatten(x1, expand_composites=True)

[<tf.Tensor: shape=(), dtype=string, numpy=b'd1'>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>]

In [73]:
import pyBigWig
print(pyBigWig.__verison__)

AttributeError: module 'pyBigWig' has no attribute '__verison__'